In [2]:
import fasttext
import pandas as pd
import numpy as np
import matplotlib as mpl
import nltk,re,pprint
import sys,glob,os
from imp import reload
from nltk.corpus import stopwords

In [2]:
 dataAnalyst = pd.read_csv("../datasets/DataAnalyst.csv", encoding="latin")

In [3]:
len(dataAnalyst["Industry"].unique())

89

In [4]:
dataAnalystSmall = dataAnalyst[['Job Description', 'Company Name', 'Industry']]

In [5]:
dataAnalystSmall

,Job Description,Company Name,Industry
0,Are you eager to roll up your sleeves and harn...,Vera Institute of Justice\n3.2,Social Assistance
1,Overview\n\nProvides analytical and technical ...,Visiting Nurse Service of New York\n3.8,Health Care Services & Hospitals
2,Weâre looking for a Senior Data Analyst who ...,Squarespace\n3.4,Internet
3,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Celerity\n4.1,IT Services
4,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,FanDuel\n3.9,Sports & Recreation
...,...,...,...
2248,Maintains systems to protect data from unautho...,"Avacend, Inc.\n2.5",Staffing & Outsourcing
2249,Position:\nSenior Data Analyst (Corporate Audi...,Arrow Electronics\n2.9,Wholesale
2250,"Title: Technical Business Analyst (SQL, Data a...",Spiceorb,-1
2251,Summary\n\nResponsible for working cross-funct...,Contingent Network Services\n3.1,Enterprise Software & Network Solutions


In [6]:
dataAnalystGrouped = dataAnalystSmall.groupby(['Industry']).count()

In [7]:
pd.set_option('display.max_rows', 500)
print(dataAnalystGrouped.sort_values(by=['Job Description'], ascending=False))

                                           Job Description  Company Name
Industry                                                                
-1                                                     353           352
IT Services                                            325           325
Staffing & Outsourcing                                 323           323
Health Care Services & Hospitals                       151           151
Consulting                                             111           111
Computer Hardware & Software                           111           111
Investment Banking & Asset Management                   78            78
Enterprise Software & Network Solutions                 69            69
Internet                                                65            65
Banks & Credit Unions                                   51            51
Advertising & Marketing                                 51            51
Insurance Carriers                                 

In [8]:
dataAnalystGrouped.shape

(89, 2)

In [9]:
dataAnalystTopCat = dataAnalystSmall[dataAnalystSmall["Industry"].isin(['IT Services', 'Staffing & Outsourcing', 'Health Care Services & Hospitals', 'Consulting',
'Computer Hardware & Software', 'Investment Banking & Asset Management', 'Enterprise Software & Network Solutions', 'Internet',
'Banks & Credit Unions', 'Advertising & Marketing'])]

In [10]:
dataAnalystTopCatGrouped = dataAnalystTopCat.groupby(['Industry'])['Job Description'].apply(lambda x: ','.join(x)).reset_index()

In [11]:
print(dataAnalystTopCatGrouped.shape)
dataAnalystTopCatGrouped.reset_index()

(10, 2)


,index,Industry,Job Description
0,0,Advertising & Marketing,About Known\n\nKnown is a modern marketing com...
1,1,Banks & Credit Unions,"Job Description:\n\nWithin GMC, Data & Portfol..."
2,2,Computer Hardware & Software,"If interested, please reach out to Iram Shah e..."
3,3,Consulting,About Us:\n\nNYSTEC is a non-profit technology...
4,4,Enterprise Software & Network Solutions,"Data Analyst Jersey City, NJ 12+ months contra..."
5,5,Health Care Services & Hospitals,Overview\n\nProvides analytical and technical ...
6,6,IT Services,Requisition NumberRR-0001939\nRemote:Yes\nWe c...
7,7,Internet,Weâre looking for a Senior Data Analyst who ...
8,8,Investment Banking & Asset Management,About Cubist\nCubist Systematic Strategies is ...
9,9,Staffing & Outsourcing,Title: Data Analyst\n\nPosition: 1+ Year Contr...


In [12]:
dataAnalystTopCatGrouped["Job Description"].shape

(10,)

In [13]:
#remove punctuations

class rempunct:
    def __init__(self):
        from nltk.corpus import stopwords
        self.swords = set(stopwords.words('english'))
        print(len(self.swords),"stopwords present!")
    
    def rem_stop_punct(self,originalText, ofilename):
        splittedText = originalText.split()
        lenl = len(splittedText)
        print("Length is: ",lenl, splittedText[:5])
        ofile = open(ofilename,'a')
        
        for r in range(lenl):
            linex = splittedText[r]
            linex2 = "".join(c for c in linex if c not in ('!','.',':',',','?',';','``','&','-','"','(',')','[',']','0','1','2','3','4','5','6','7','8','9'))
            linex3 = linex2.split()
            #prog=(r+1)/len(rawlines)
            for s in range(len(linex3)):
                noword = linex3[s].lower()
                if noword not in self.swords:
                    ofile.write(noword)
                    ofile.write(" ") 

    def drawProgressBar(self,percent, barLen = 50):			#just a progress bar so that you dont lose patience
        sys.stdout.write("\r")
        progress = ""
        for i in range(barLen):
            if i<int(barLen * percent):
                progress += "="
            else:
                progress += " "
        sys.stdout.write("[ %s ] %.2f%%" % (progress, percent * 100))
        sys.stdout.flush()

In [14]:
rp = rempunct()
for index, row in dataAnalystTopCatGrouped.iterrows():
    ofname = '../processFiles/trainCatFiles/cat' + str(index)+ '_' + '-'.join(row['Industry'].split()) + '.txt'    
    print("Processing...",row['Industry'])
    rp.rem_stop_punct(row['Job Description'],ofname)

179 stopwords present!
Processing... Advertising & Marketing
Length is:  21227 ['About', 'Known', 'Known', 'is', 'a']
Processing... Banks & Credit Unions
Length is:  32003 ['Job', 'Description:', 'Within', 'GMC,', 'Data']
Processing... Computer Hardware & Software
Length is:  50038 ['If', 'interested,', 'please', 'reach', 'out']
Processing... Consulting
Length is:  48362 ['About', 'Us:', 'NYSTEC', 'is', 'a']
Processing... Enterprise Software & Network Solutions
Length is:  35454 ['Data', 'Analyst', 'Jersey', 'City,', 'NJ']
Processing... Health Care Services & Hospitals
Length is:  84819 ['Overview', 'Provides', 'analytical', 'and', 'technical']
Processing... IT Services
Length is:  96150 ['Requisition', 'NumberRR-0001939', 'Remote:Yes', 'We', 'collaborate.']
Processing... Internet
Length is:  34604 ['Weâ\x80\x99re', 'looking', 'for', 'a', 'Senior']
Processing... Investment Banking & Asset Management
Length is:  41725 ['About', 'Cubist', 'Cubist', 'Systematic', 'Strategies']
Processing.

In [3]:
# Term Frequency(TF)
from collections import Counter
import operator
import os
import sys
import string
import argparse
import numpy as np

def getDocumentTF(fname):
    #file1 = open("../processFiles/trainCatFiles/cat2_Computer-Hardware-&-Software.txt","r")
    file1 = open(fname)
    readfile = file1.read()
    wordsInFile = readfile.split()
    counts_all = Counter(wordsInFile)
    words, count_values = zip(*counts_all.items())
    # Sort both lists by frequency in values (Schwartzian transform) - thanks, http://stackoverflow.com/questions/9543211/sorting-a-list-in-python-using-the-result-from-sorting-another-list
    values_sorted, words_sorted = zip(*sorted(zip(count_values, words), key=operator.itemgetter(0), reverse=True))
    # Top N
    #top_n = 250
    #words_sorted_top = words_sorted[0:top_n]
    #values_sorted_top = values_sorted[0:top_n]
    countdf = pd.DataFrame({'count': values_sorted, 'word': words_sorted})
    return countdf
    
#def processAllDocuments():
owd = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN"
print(owd)
fld = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/"
os.chdir(fld)
arrTermFreq = []
for file in glob.glob("*.txt"):
    print("found",file)
    doctf = getDocumentTF(file)
    arrTermFreq.append(doctf)
os.chdir(owd)
len(arrTermFreq)

/Users/arnabborah/Documents/repositories/textclusteringDBSCAN
found cat5_Health-Care-Services-&-Hospitals.txt
found cat6_IT-Services.txt
found cat0_Advertising-&-Marketing.txt
found cat7_Internet.txt
found cat4_Enterprise-Software-&-Network-Solutions.txt
found cat1_Banks-&-Credit-Unions.txt
found cat3_Consulting.txt
found cat9_Staffing-&-Outsourcing.txt
found cat2_Computer-Hardware-&-Software.txt
found cat8_Investment-Banking-&-Asset-Management.txt


10

In [18]:
def getTermFreq(docId,term):
    valx = arrTermFreq[docId][arrTermFreq[docId]['word']==term]['count'].tolist()
    if len(valx)==0:
        return 0
    return valx[0]

def drawProgressBar(percent, barLen = 50):			#just a progress bar so that you dont lose patience
    sys.stdout.write("\r")
    progress = ""
    for i in range(barLen):
        if i<int(barLen * percent):
            progress += "="
        else:
            progress += " "
    sys.stdout.write("[ %s ] %.2f%%" % (progress, percent * 100))
    sys.stdout.flush()	

In [19]:
for j in range(1,9):
    b = getTermFreq(j,"candidate")
    print(b)

54
20
20
5
37
40
91
29


In [73]:
#Inverse Document Frequency(IDF)

import math

owd = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN"
print(owd)
fld = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/"
os.chdir(fld)
termsForIDF = []
totalwords = 0 
for file in glob.glob("*.txt"):
    print("found",file)    
    file1 = open(file)
    readfile = file1.read()
    wordsInFile = readfile.split()
    lenx=len(wordsInFile)
    totalwords+=lenx
    termsForIDF.extend(set(wordsInFile))
print(len(termsForIDF),totalwords)

def getIdf(term):
    countPresentDocs = 0
    for i in range(len(arrTermFreq)):
        tfx = getTermFreq(i,term)
        #print(tfx)
        if tfx>0:
            countPresentDocs+=1
            #print("found one")
    #print(countPresentDocs,"countPresentDocs")
    return countPresentDocs
    
    return logg1

#idfMatrix = {'term':'Zara','idfx':'1'}
idfMatrix = {}
lenv = len(termsForIDF)
#lenv = 100
print("Computing inverse document frequencies")
for j in range(lenv):
    el = termsForIDF[j]
    idfx = getIdf(el)
    idfy = lenv/float(1+idfx)
    idfz = math.log(idfy,10)
    idfMatrix[el] = [idfz]
    prog=(j+1)/lenv
    drawProgressBar(prog)
    
#pd.DataFrame(idfMatrix)
idfMatrixDF = pd.DataFrame.from_dict(idfMatrix, orient = 'index')

/Users/arnabborah/Documents/repositories/textclusteringDBSCAN
found cat5_Health-Care-Services-&-Hospitals.txt
found cat6_IT-Services.txt
found cat0_Advertising-&-Marketing.txt
found cat7_Internet.txt
found cat4_Enterprise-Software-&-Network-Solutions.txt
found cat1_Banks-&-Credit-Unions.txt
found cat3_Consulting.txt
found cat9_Staffing-&-Outsourcing.txt
found cat2_Computer-Hardware-&-Software.txt
found cat8_Investment-Banking-&-Asset-Management.txt
50605 375812
Computing inverse document frequencies
[ ================================================== ] 100.00%

In [74]:
owd = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN"
os.chdir(owd)
idfMatrixDF.to_csv('processFiles/idfMatrixDF.csv', header=False)

In [104]:
#tfidf

class flingTFIDF:
    def __init__(self):
        self.nof = 0
        self.tfidfMatrix = []
        #allfnames = self.allfiles("/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles")

    def compute_tfidf(self,fileIndex,filename):
        tfidfFileMatrix = {}
        file1 = open(filename)
        readfile = file1.read()
        wordsInFile = set(readfile.split())
        for word in wordsInFile:
            tf_final = getTermFreq(fileIndex,word)
            idf_final = idfMatrix[word][0]
            tfidf_final = tf_final * idf_final
            tfidfFileMatrix[word] = tfidf_final
        return tfidfFileMatrix
        
    def getallfilenames(self,foldername):#returns the name of all files inside the source folder. 		
        owd = os.getcwd()
        fld = foldername + "/"
        os.chdir(fld)					#this is the name of the folder from which the file names are returned.
        fnamearr = []						#empty array, the names of files are appended to this array, and returned.
        for file in glob.glob("*.txt"):
            fnamearr.append(file)
        os.chdir(owd)
        return fnamearr
        
    def computeTFIDFallfiles(self,flist):
        for fin in range(len(flist)):
            fname = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/" + flist[fin]
            print("file:",fname)
            tfidfMAT = self.compute_tfidf(fin,fname)
            self.tfidfMatrix.append(tfidfMAT)
        print("length of tfidfMatrix:",len(self.tfidfMatrix))        

In [105]:
ft = flingTFIDF()
allfnames = ft.getallfilenames("/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles")
ft.computeTFIDFallfiles(allfnames)

file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat5_Health-Care-Services-&-Hospitals.txt
file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat6_IT-Services.txt
file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat0_Advertising-&-Marketing.txt
file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat7_Internet.txt
file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat4_Enterprise-Software-&-Network-Solutions.txt
file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat1_Banks-&-Credit-Unions.txt
file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat3_Consulting.txt
file: /Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/trainCatFiles/cat9_Staffing-&-Outsourcin

In [106]:
print(len(ft.tfidfMatrix))
print(len(idfMatrix))

10
19483


In [108]:
# save characteristic files for categories assigned 
fwd = "/Users/arnabborah/Documents/repositories/textclusteringDBSCAN/processFiles/characteristicFiles"
os.chdir(fwd)
print(len(ft.tfidfMatrix))
for matrixID in range(len(ft.tfidfMatrix)):
    fname_orig = allfnames[matrixID]
    newfname = fname_orig.split()[0]+'_characteristic.csv'
    print("filename:",newfname)
    matrixDF = pd.DataFrame.from_dict(ft.tfidfMatrix[matrixID], orient = 'index')
    matrixDF.to_csv(newfname, header=False)
os.chdir("/Users/arnabborah/Documents/repositories/textclusteringDBSCAN")

10
filename: cat5_Health-Care-Services-&-Hospitals.txt_characteristic.csv
filename: cat6_IT-Services.txt_characteristic.csv
filename: cat0_Advertising-&-Marketing.txt_characteristic.csv
filename: cat7_Internet.txt_characteristic.csv
filename: cat4_Enterprise-Software-&-Network-Solutions.txt_characteristic.csv
filename: cat1_Banks-&-Credit-Unions.txt_characteristic.csv
filename: cat3_Consulting.txt_characteristic.csv
filename: cat9_Staffing-&-Outsourcing.txt_characteristic.csv
filename: cat2_Computer-Hardware-&-Software.txt_characteristic.csv
filename: cat8_Investment-Banking-&-Asset-Management.txt_characteristic.csv
